In [4]:
# !pip install selenium

In [5]:
# !pip install --upgrade pip

In [6]:
# !pip install webdriver_manager

In [3]:
# !pip3 install bs4

In [7]:
# Import all required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import requests
import re
import matplotlib as pl
import matplotlib.pyplot as plt

In [10]:
# Create an instance of chrome browser using the chrome driver
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Maximize the Chrome window
driver.maximize_window()

# Interact with ebay website
driver.get('https://www.amazon.in/')
time.sleep(2)

In [11]:
#Navigate to Televisions page
element_elc = driver.find_element(By.XPATH, "//a[text()=' Electronics ']")
element_elc.click()
time.sleep(3)

element_elc = driver.find_element(By.XPATH, "//span[text()='Home Theater, TV & Video']")
element_elc.click()
time.sleep(3)

element_tel = driver.find_element(By.XPATH, "//span[text()='Televisions']")
element_tel.click()
time.sleep(3)


In [12]:
#apply filter to select top brands
element_tv_size = driver.find_element(By.XPATH, "//span[@class='a-size-base a-color-base' and text()='Top Brands']")
element_tv_size.click()
time.sleep(3)


In [13]:
# load page data and return prodtuct details
def get_product_details():
    soup = BeautifulSoup(driver.page_source,'html.parser')
    soup_list = soup.find_all(class_ = "s-main-slot s-result-list s-search-results sg-row")
    
    # Create a BeautifulSoup object from the returned resutlset
    soup = BeautifulSoup(str(soup_list), 'html.parser')
    # Find all television items
    return soup.find_all('div', class_='s-result-item')

In [14]:
#Store the results obtained into a Dataframe and save it as CSV file. Two coloumns, product_name and product_price.
# Title column : Product Name, Price
try:
    products = []
    prices = []

    items = get_product_details()
    # Check if there is a next page and collect all product details into items list
    xpath_nextpage = "//a[text() = 'Next']"
    while(len(driver.find_elements(By.XPATH, xpath_nextpage)) > 0):
        element_nextpage = driver.find_elements(By.XPATH, xpath_nextpage)
        driver.execute_script("arguments[0].scrollIntoView();",element_nextpage[0])
        element_nextpage[0].click()
        time.sleep(3)
        items.extend(get_product_details())
        
    # print(items)
    # Iterate over the product list and extract the name, and price
    for item in items:
        name = item.find('span', class_='a-size-base-plus a-color-base a-text-normal')
        price = item.find('span', class_='a-price-whole')
        if(name != None and price != None):
            name = name.text.strip()
            price = 'INR ' + price.text.strip()
            products.append(name)
            prices.append(price)
            
    data = {'Product Name':products, 'Price':prices}
    df = pd.DataFrame(data)
    driver.close()
    
except Exception as e:
    print('An error occurred during scraping:', str(e))

In [15]:
#Clean up the raw data
df = df.drop_duplicates()
print(df.__len__())

170


In [16]:
# Define the regular expression pattern for the brands
pattern = r'(LG|Samsung|Sony|TCL|Hisense|Panasonic|Sharp|Philips|Haier|Toshiba|MI|Redmi|VW|iFFALCON|Acer|OnePlus|Vu|Xiaomi|Blaupunkt|Kodak|Croma|Westinghouse|Onida|Dyanora|Sansui)'

# Extract the brand from the 'Product Name' column and assign it to the 'Brand' column
df['Brand'] = df['Product Name'].str.extract(pattern, flags=re.IGNORECASE, expand=False)

# Convert the 'Brand' column to uppercase
df['Brand'] = df['Brand'].str.upper()

In [17]:
# Convert the column to string type
df['Product Name'] = df['Product Name'].astype(str)

# Define the regular expression pattern for the size
pattern = r"(\d+)\s*(inch|inches|Inch)"
matches = df['Product Name'].apply(lambda x: re.findall(pattern, x))

# Extract the matches and assign to the 'Size' column
df['Size'] = [', '.join(match[0]) if match else None for match in matches]
df['Size'] = df['Size'].replace(', inch|, Inch', '', regex=True)
df.drop(df[df['Size'].isnull()].index, inplace=True)
 
# Sorting by column 'Size'
df.sort_values(by=['Size'], inplace=True)


In [18]:
# Define the regular expression pattern for the display
pattern = r'(LED|QLED|OLED)'

# Extract the display from the 'Product Name' column and assign it to the 'Display' column
df['Display'] = df['Product Name'].str.extract(pattern, flags=re.IGNORECASE, expand=False)

# Convert the 'Display' column to uppercase
df['Display'] = df['Display'].str.upper()
df['Display'].fillna('LED', inplace=True)

C:\Users\vasu_seelamneni\AppData\Local\Temp\ipykernel_30140\203849084.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Display'].fillna('LED', inplace=True)


In [19]:
df.to_csv('..\\..\\miniproject\\data\\amazon_television_data.csv', index=False)